In [ ]:
from langgraph.graph import MessagesState, START, END, StateGraph
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage, SystemMessage, AnyMessage, filter_messages
from langchain.chat_models import init_chat_model

from typing import Literal, Annotated
from pydantic import Field
import asyncio

from sys import path

path.append("src/")
from utils import get_buffer_string
from prompts import compress_research_system_prompt, compress_research_human_message, current_state_instructions, future_state_instructions
from tools.others import get_today_str
from subagents.research_lead_agent import ResearchLeadAgent
from tools.think import think_tool
from tools.supervise import ResearchComplete, ConductResearch


# graph state
class MacroAgentState(MessagesState):
    user_query: str
    current_state_messages: Annotated[list[AnyMessage], add_messages] = Field(default=list)
    current_state_summary: str
    future_events_messages: Annotated[list[AnyMessage], add_messages] = Field(default=list)
    future_events_summary: str


# graph flow
class GetCurrentState:

    def __init__(self, llm_config, tools):
        self._llm_config = llm_config
        model = init_chat_model(
            model=self._llm_config.get("model_name"), 
            temperature=self._llm_config.get("temperature")
        )
        self.llm = model.bind_tools(tools)

    async def __call__(self, state):

        if not state.get("current_state_messages"):
            state.get("current_state_messages").append(HumanMessage(content=state.get("user_query")))

        messages = state.get("current_state_messages")
        
        response = await self.llm.ainvoke(
            input=[
                HumanMessage(content=current_state_instructions.format(
                    messages=get_buffer_string(messages),
                    date=get_today_str()
                ))
            ]
        )

        print("GetCurrentState", response)

        return {"current_state_messages": [response], "messages": [response]}


class GetFutureEvents:

    def __init__(self, llm_config, tools):
        self._llm_config = llm_config
        model = init_chat_model(
            model=self._llm_config.get("model_name"), 
            temperature=self._llm_config.get("temperature")
        )
        self.llm = model.bind_tools(tools)

    async def __call__(self, state):

        if not state.get("future_events_messages"):
            state["future_events_messages"].extend(
                [
                    HumanMessage(content=future_state_instructions.format(
                        messages="",
                        date=get_today_str(),
                        current_state_summary=state.get("current_state_summary")
                    ))
                ]
            )

        messages = state.get("future_events_messages")
        
        response = await self.llm.ainvoke(
            input=messages
        )
        print("GetFutureEvents", response)
        
        return {"future_events_messages": [response], "messages": [response]}


class Summarizer:

    def __init__(self, llm_config, node_before: Literal["current_state", "future_events"]):
        self._llm_config = llm_config
        model = init_chat_model(
            model=self._llm_config.get("model_name"), 
            temperature=self._llm_config.get("temperature")
        )
        self.llm = model
        self._node_before = node_before

    async def __call__(self, state: MacroAgentState):
        """Compress research findings into a concise summary.
        
        Takes all the research messages and tool outputs and creates
        a compressed summary suitable for the supervisor's decision-making.
        """
        
        system_message = compress_research_system_prompt.format(date=get_today_str())

        messages_key = "current_state_messages" if self._node_before == "current_state" else "future_events_messages"
        messages = [SystemMessage(content=system_message)] + state.get(messages_key, []) + [HumanMessage(content=compress_research_human_message)]
        response = await self.llm.ainvoke(messages)
               
        summary_key = "current_state_summary" if self._node_before == "current_state" else "future_events_summary"
        return {
            summary_key: str(response.content)
        }


class ToolNode:
    
    def __init__(self, tools, research_tool, macro_step: Literal["current_state", "future_events"]):

        self.research_tool = research_tool
        self.tools_by_name = {tool.name: tool for tool in tools}
        self._macro_step = macro_step

    async def __call__(self, state, config):
        """Execute supervisor decisions - either conduct research or end the process.
        
        Handles:
        - Executing think_tool calls for strategic reflection
        - Launching parallel research agents for different topics
        - Aggregating research results
        - Determining when research is complete
        
        Args:
            state: Current supervisor state with messages and iteration count
            
        Returns:
            Command to continue supervision, end process, or handle errors
        """
        messages_key = "current_state_messages" if self._macro_step == "current_state" else "future_events_messages"
        messages = state.get(messages_key, [])
        most_recent_message = messages[-1]
        
        # Initialize variables for single return pattern
        tool_messages = []

        try:

            any_complete_research = [
                tool_call for tool_call in most_recent_message.tool_calls 
                if tool_call["name"] == "ResearchComplete"
            ]

            if any_complete_research:
                tool_results = [{'content': "Research Completed" for m in any_complete_research}]
                tool_messages.extend([
                        ToolMessage(
                            content=result.get("content"),
                            name=tool_call["name"],
                            tool_call_id=tool_call["id"]
                        ) for result, tool_call in zip(tool_results, any_complete_research)
                    ])
            else:
                # Separate think_tool calls from ConductResearch calls
                think_tool_calls = [
                    tool_call for tool_call in most_recent_message.tool_calls 
                    if tool_call["name"] == "think_tool"
                ]
                
                conduct_research_calls = [
                    tool_call for tool_call in most_recent_message.tool_calls 
                    if tool_call["name"] == "ConductResearch"
                ]

                # Handle think_tool calls (synchronous)
                for tool_call in think_tool_calls:
                    observation = self.tools_by_name["think_tool"].invoke(tool_call["args"])
                    tool_messages.extend([
                        ToolMessage(
                            content=observation,
                            name=tool_call["name"],
                            tool_call_id=tool_call["id"]
                        )
                    ])

                # Handle ConductResearch calls (asynchronous)
                if conduct_research_calls:
                    # Launch parallel research agents
                    coros = [
                        self.research_tool.ainvoke(input={
                            "messages": [
                                HumanMessage(content=tool_call["args"]["research_topic"])
                            ],
                            "research_topic": tool_call["args"]["research_topic"]
                        },
                        config=config) 
                        for tool_call in conduct_research_calls
                    ]

                    # Wait for all research to complete
                    tool_results = await asyncio.gather(*coros)
                    # tool_results = [{"research_summary": "US economy is fine"} for t in conduct_research_calls]

                    # Format research results as tool messages
                    # Each sub-agent returns compressed research findings in result["compressed_research"]
                    # We write this compressed research as the content of a ToolMessage, which allows
                    # the supervisor to later retrieve these findings via get_notes_from_tool_calls()
                    research_tool_messages = [
                        ToolMessage(
                            content=result.get("research_summary", "Error synthesizing research report"),
                            name=tool_call["name"],
                            tool_call_id=tool_call["id"]
                        ) for result, tool_call in zip(tool_results, conduct_research_calls)
                    ]
                    
                    tool_messages.extend(research_tool_messages)

            return {'messages': tool_messages, messages_key: tool_messages}
                    
        except Exception as e:
            print(f"Error in supervisor tools: {e}")


def continue_current_state_search_or_pass_to_future_events_search(state: MacroAgentState):

    last_message = state.get("current_state_messages")[-1]

    if isinstance(last_message, AIMessage):
        path = "current_state_summarizer" if any(
            tool_call["name"] == "ResearchComplete" 
            for tool_call in last_message.tool_calls
            
        ) else "current_state"
    elif isinstance(last_message, ToolMessage):
        path = "current_state_summarizer" if last_message.name == "ResearchComplete" else "current_state"
    return path


def continue_future_events_search_or_end(state: MacroAgentState):

    last_message = state.get("messages")[-1]
    if isinstance(last_message, AIMessage):
        path = "future_events_summarizer" if any(
            tool_call["name"] == "ResearchComplete" 
            for tool_call in last_message.tool_calls
            
        ) else "future_events"
    elif isinstance(last_message, ToolMessage):
        path = "future_events_summarizer" if last_message.name == "ResearchComplete" else "future_events"

    return path


class MacroAgent:

    def __init__(self, llm_config, compile_config):
        self.llm_config = llm_config
        self.compile_config = compile_config
        self.graph = None
        self.compiled_graph = None
        self._research_lead_agent = ResearchLeadAgent(llm_config=self.llm_config, compile_config=self.compile_config)
        self.tools = [
            ConductResearch, ResearchComplete, think_tool
        ]

        self._build_graph()
        self._compile_graph()

    def _build_graph(self):

        graph = StateGraph(MacroAgentState)
        graph.add_node("current_state", GetCurrentState(llm_config=self.llm_config.get("planner").get("current_state"), tools=self.tools))
        graph.add_node("future_events", GetFutureEvents(llm_config=self.llm_config.get("planner").get("future_events"), tools=self.tools))
        graph.add_node("current_state_tools", ToolNode(tools=self.tools, research_tool=self._research_lead_agent, macro_step="current_state"))
        graph.add_node("future_events_tools", ToolNode(tools=self.tools, research_tool=self._research_lead_agent, macro_step="future_events"))
        graph.add_node("current_state_summarizer", Summarizer(llm_config=self.llm_config.get("research").get("summarize_research"), node_before="current_state"))
        graph.add_node("future_events_summarizer", Summarizer(llm_config=self.llm_config.get("research").get("summarize_research"), node_before="future_events"))

        graph.add_edge(START, "current_state")
        graph.add_edge("current_state", "current_state_tools")
        graph.add_conditional_edges("current_state_tools", continue_current_state_search_or_pass_to_future_events_search, {"current_state_summarizer": "current_state_summarizer", "current_state": "current_state"})
        graph.add_edge("current_state_summarizer", "future_events")
        graph.add_edge("future_events", "future_events_tools")
        graph.add_conditional_edges("future_events_tools", continue_future_events_search_or_end, {"future_events_summarizer": "future_events_summarizer", "future_events": "future_events"})
        graph.add_edge("future_events_summarizer", END)

        self.graph = graph
    
    def _compile_graph(self):
        self.compiled_graph = self.graph.compile(**self.compile_config)
    
    async def ainvoke(self, input, config):
        return await self.compiled_graph.ainvoke(input, config=config)
    
    async def __call__(self, input, config):
        return await self.ainvoke(input, config)
    

In [2]:
from sys import path
import yaml
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import InMemorySaver

path.append("../src/")

from utils import format_messages
# from app import MacroAgent

# compilation config
checkpointer = InMemorySaver()
compile_config = {'checkpointer': checkpointer}

# llm config
llm_config = yaml.safe_load(open("config/llm.yaml", "rb"))

# app
app = MacroAgent(
    llm_config=llm_config,
    compile_config=compile_config
)


In [3]:
# from IPython.display import Image, display

# display(Image(app.compiled_graph.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
config = {"configurable": {"thread_id": "scoping-session-0", "recursion_limit": 100}}
user_query = "What is the health of the current US economy?"
result = await app.ainvoke(
    input={"user_query": user_query, "messages": [HumanMessage(content=user_query)]},
    config=config
)
format_messages(result['messages'])

[HumanMessage(content='What is the health of the current US economy?', additional_kwargs={}, response_metadata={})]
GetCurrentState content='' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 617, 'total_tokens': 642, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b547601dbd', 'id': 'chatcmpl-ChxT1cxmhmF7XvcNtrRmQFErWpcBX', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='lc_run--f1c3b476-c8aa-4bfb-b194-ecef44f482fd-0' tool_calls=[{'name': 'ConductResearch', 'args': {'research_topic': 'current status of the US economy December 2025'}, 'id': 'call_rulxz3YmPyTlxMHBldn0brO0', 'type': 'tool_call'}] usage_metadata={'input_tokens': 617, 'output_token

CancelledError: 

Failed to summarize webpage: Error code: 400 - {'error': {'message': "This model's maximum context length is 128000 tokens. However, you requested 129733 tokens (113349 in the messages, 16384 in the completion). Please reduce the length of the messages or completion.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}
Failed to summarize webpage: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-Tn6esdNG8gt3NcFOQNHo81m2 on tokens per min (TPM): Limit 200000, Used 141677, Requested 174404. Please try again in 34.824s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}


In [ ]:
from rich.console import Console
from rich.markdown import Markdown

console = Console()
md = Markdown(result["current_state_summary"])
console.print(md)

List of Queries and Tool Calls Made                                                                                

 • Research on the current status of the US economy as of December 2025.                                           

Fully Comprehensive Findings The US economy is fine.                                                               

List of All Relevant Sources (with citations in the report) [1] Source Title: Current Status of the US Economy -   
December 2025: URL

In [ ]:
console = Console()
md = Markdown(result["future_events_summary"])
console.print(md)

List of Queries and Tool Calls Made                                                                                

 • Research on the current status of the US economy as of December 2025.                                           
 • Upcoming events that might affect the US economy in 2026.                                                       
 • Political events affecting the US economy in 2026.                                                              
 • Federal Reserve meetings schedule 2026.                                                                         
 • International trade agreements impacting US economy in 2026.                                                    
 • Major economic reports due in 2026.                                                                             

Fully Comprehensive Findings The US economy is fine as of December 2025. However, searches for upcoming events that
might affect the US economy in 2026 did not yield specific information. Multiple searches were conducted focusing  
on various areas that could impact the economy, including political events, Federal Reserve meetings, international
trade agreements, and major economic reports. Unfortunately, all searches returned the same result, indicating a   
lack of specific information available at this time.                                                               

List of All Relevant Sources (with citations in the report) [1] Source Title: Current Status of the US Economy -   
December 2025: URL